# 7.3 PDE-Constrained Shape Optimization (semi-automated)

We want to solve the PDE-constrained shape optimization problem
$$
            \underset{\Omega\subset \mathsf{D}}{\mbox{min}} \; J(u) := \int_\Omega |u-u_d|^q \; dx, \quad q\ge 2
$$
 subject to that $(\Omega,u)$ satisfy
 $$
           \int_\Omega \nabla u \cdot \nabla v \; dx = \int_\Omega f v \; dx \; \quad \text{ for all } v \in H_0^1(\Omega),
$$
where $\Omega \subset \mathbb R^2$ for given $u_d, \, f \in C^1(\mathbb R^2)$. 

Here, we want to compute the shape derivative by differentiation of a suitably defined perturbed Lagrangian using automated differentiation. For details we refer to 

P. Gangl, K. Sturm, M. Neunteufel, J. Schöberl.
Fully and Semi-Automated Shape Differentiation in NGSolve,
Struct. Multidisc. Optim., 63, pp.1579-1607, 2021.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry

In [2]:
geo = SplineGeometry()
geo.AddCircle(c=(0.5,0.5), r=0.5, bc = 'circle')
mesh = Mesh(geo.GenerateMesh(maxh = 0.08))

In [3]:
#given data of our problem (chosen such that \Omega^* = [0,1]^2 is the optimal shape)
f = CoefficientFunction(2*y*(1-y)+2*x*(1-x))
ud = x*(1-x)*y*(1-y)

grad_f = CoefficientFunction( (f.Diff(x), f.Diff(y) ) )
grad_ud = CoefficientFunction( (ud.Diff(x), ud.Diff(y) ) )

### State equation

In [4]:
fes = H1(mesh, order=2, dirichlet=".*")
u, v = fes.TnT()
gfu = GridFunction(fes)
scene = Draw(gfu, mesh, "state")

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx

fstate = LinearForm(fes)
fstate += f*v*dx

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [5]:
def SolveStateEquation():
    rhs = gfu.vec.CreateVector()
    rhs.data = fstate.vec - a.mat * gfu.vec
    update = gfu.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()) * rhs
    gfu.vec.data += update

In [20]:
a.Assemble()
fstate.Assemble()
SolveStateEquation()

scene.Redraw()

### Adjoint equation

We set up the adjoint equation
$$
    \mbox{Find } p \in H_0^1(\Omega): \int_\Omega \nabla w \cdot \nabla p \, \mbox dx = - \partial_u J(u)(w) \quad \text{ for all } w \in H_0^1(\Omega)
$$
where $u$ is the solution to the state equation. For $J(u) = \int_\Omega |u-u_d|^q \mbox dx$, we get
$$
    \partial_u J(u)(w) = q \int_\Omega (u-u_d)^{d-1}w \,\mbox dx.
$$
However, we can also use the Diff(...) command:

In [21]:
q=4
def Cost(u): 
    return (u-ud)**q*dx

p, w = fes.TnT()
gfp = GridFunction(fes)
scene = Draw (gfp, mesh, "adjoint")

fadjoint = LinearForm(fes)
fadjoint += -1*Cost(gfu).Diff(gfu,w)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [22]:
def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - a.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()).T * rhs
    gfp.vec.data += update

In [23]:
fadjoint.Assemble()
SolveAdjointEquation()
scene.Redraw()

Note that (for linear problems) the operator on the left hand side of the adjoint equation is just the transpose of the state operator.

### Automatic Shape Differentiation

The formula for the shape derivative was derived as the partial derivative of the perturbed Lagrangian (brought back to the original domain):
$$
    d\mathcal J(\Omega; X) = \frac{\partial}{\partial t} \left. \left( \int_\Omega \xi(t)|u - u_d^t|^q \mbox{d} x 
         +  \int_{\Omega} (F_t^{-\top}\nabla u) \cdot (F_t^{-\top} \nabla p) \xi(t) \, dx - \int_{\Omega} \xi(t) f^t p \,dx \right) \right\rvert_{t=0} 
$$
where 
<ul>
    <li>   $T_t(x)=x+tX(x)=y$ 
    <li> $F_t = DT_t = I+t DX$
    <li> $\xi(t) = \mbox{det}(F_t)$
    <li>  $u_d^t = u_d \circ T_t$
    <li> $f^t = f \circ T_t$
</ul>

The integrand depends on the parameter $t$ only via $\xi(t)$, $F_t$ and $T_t$. Denoting the integrand by $G^{u,p}$, the derivative is given by
$$
\begin{array}{rl}
     d\mathcal J(\Omega; X) =& \frac{d G^{u,p}}{d \xi} \frac{d \xi}{d t} + \frac{d  G^{u,p}}{d F} \frac{d F}{d t} + \frac{d  G^{u,p}}{dy} \cdot \frac{d T_t}{dt} \\
     =& \frac{d G^{u,p}}{d \xi} \mbox{div}(X) + \frac{d  G^{u,p}}{d F} DX + \frac{d  G^{u,p}}{dy} \cdot X
\end{array}
$$

We define the perturbed PDE and cost function involving parameters $F$ and $J$ representing the Jacobian of a shape transformation and its determinant, respectively. While their values are set to unity and thus does not influence the solution of the state or adjoint equation, this procedure allows us to differentiate with respect to them.

In [24]:
J = Parameter(1)
F = Id(2)
#Finv = 2*Id(2) - F    # consistent linearization at F = I (avoids calling Inv() for speedup)

def Equation(u,v):
    return ( (Inv(F.trans)*grad(u))*(Inv(F.trans)*grad(v))-f*v)*J*dx
#    return ( (Finv.trans*grad(u))*(Finv.trans*grad(v))-f*v)*J*dx

def CostAuto(u): 
    return J*(u-ud)**q*dx

In [25]:
VEC = H1(mesh, order=2, dim=2)
PHI, X = VEC.TnT()

Lagrangian = CostAuto(gfu) + Equation(gfu,gfp)
dJOmegaAuto = LinearForm(VEC)
dJOmegaAuto += Lagrangian.Diff(J, div(X))
dJOmegaAuto += Lagrangian.Diff(F, grad(X).trans)   # grad(X).trans is Jacobian
dJOmegaAuto += Lagrangian.Diff(x, X[0]) + Lagrangian.Diff(y, X[1])

In [26]:
b = BilinearForm(VEC)
b += InnerProduct(grad(X),grad(PHI))*dx + InnerProduct(X,PHI)*dx

gfX = GridFunction(VEC)

In [27]:
def SolveDeformationEquationAuto():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmegaAuto.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [28]:
b.Assemble()
dJOmegaAuto.Assemble()
SolveDeformationEquationAuto()
Draw(-gfX, mesh, "gfX")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

In [29]:
# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
mesh.SetDeformation(gfset)
sceneSet = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [30]:
gfset.Set((0,0))
mesh.SetDeformation(gfset)
print('Cost at initial design', Integrate (CostAuto(gfu), mesh))

scale = 0.5 / Norm(gfX.vec)
gfset.vec.data -= scale * gfX.vec
mesh.SetDeformation(gfset)
sceneSet.Redraw()

Cost at initial design 3.424780558662707e-09


In [31]:
a.Assemble()
fstate.Assemble()
SolveStateEquation()
print('Cost at new design', Integrate (CostAuto(gfu), mesh))

Cost at new design 1.0575735360481383e-08


Equation can also be used to define the bilinear form. The following defines the same bilinear form as above:

In [32]:
aAuto = BilinearForm(fes, symmetric=True)
aAuto += Equation(u,v)

Thus, the user has to enter the PDE (in its transformed form) only once.

Finally, let us again run the full algorithm:

In [33]:
#reset to and solve for initial configuration
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

a.Assemble()
fstate.Assemble()
SolveStateEquation()

LineSearch = False

iter_max = 600
Jold = Integrate(CostAuto(gfu), mesh)
converged = False
for k in range(iter_max):
    scene.Redraw()
    print('cost at iteration', k, ': ', Jold)
    mesh.SetDeformation(gfset)
    
    a.Assemble()
    fstate.Assemble()
    SolveStateEquation()
    
    fadjoint.Assemble()
    SolveAdjointEquation()
    
    b.Assemble()
    dJOmegaAuto.Assemble()
    SolveDeformationEquationAuto()

    scale = 0.01 / Norm(gfX.vec)
    gfsetOld = gfset
    gfset.vec.data -= scale * gfX.vec
    
    Jnew = Integrate(CostAuto(gfu), mesh)
    
    if LineSearch:
        while Jnew > Jold and scale > 1e-12:
            scale = scale / 2
            
            if scale <= 1e-12:
                converged = True
                break

            gfset.vec.data = gfsetOld.vec - scale * gfX.vec
            mesh.SetDeformation(gfset)
            
            a.Assemble()
            fstate.Assemble()
            SolveStateEquation()
            Jnew = Integrate(CostAuto(gfu), mesh)
    
    if converged==True:
        break
    Jold = Jnew



WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

cost at iteration 0 :  5.15224413081495e-09
cost at iteration 1 :  4.942040315052451e-09
cost at iteration 2 :  4.585080869047023e-09
cost at iteration 3 :  4.2494398855982596e-09
cost at iteration 4 :  3.934229274496257e-09
cost at iteration 5 :  3.6385740544274712e-09
cost at iteration 6 :  3.3616132927088315e-09
cost at iteration 7 :  3.102501038477645e-09
cost at iteration 8 :  2.8604072470352585e-09
cost at iteration 9 :  2.63451869268272e-09
cost at iteration 10 :  2.424039866929766e-09
cost at iteration 11 :  2.228193858360781e-09
cost at iteration 12 :  2.046223209662374e-09
cost at iteration 13 :  1.8773907462767712e-09
cost at iteration 14 :  1.720980369755032e-09
cost at iteration 15 :  1.5762978070056823e-09
cost at iteration 16 :  1.4426713040877704e-09
cost at iteration 17 :  1.3194522497306292e-09
cost at iteration 18 :  1.2060157090339143e-09
cost at iteration 19 :  1.101760841345096e-09
cost at iteration 20 :  1.0061111674975617e-09
cost at iteration 21 :  9.1851463957

cost at iteration 177 :  1.1841462497255839e-15
cost at iteration 178 :  1.1442841435162155e-15
cost at iteration 179 :  1.1070185537304874e-15
cost at iteration 180 :  1.070831382034867e-15
cost at iteration 181 :  1.0370412509444706e-15
cost at iteration 182 :  1.0042496550676848e-15
cost at iteration 183 :  9.736518554055357e-16
cost at iteration 184 :  9.439624454810348e-16
cost at iteration 185 :  9.162666770210437e-16
cost at iteration 186 :  8.893865615339492e-16
cost at iteration 187 :  8.643079784914744e-16
cost at iteration 188 :  8.399552001149714e-16
cost at iteration 189 :  8.172231407771183e-16
cost at iteration 190 :  7.951334285042868e-16
cost at iteration 191 :  7.744967529300874e-16
cost at iteration 192 :  7.544272998804951e-16
cost at iteration 193 :  7.356570713298268e-16
cost at iteration 194 :  7.173880864613703e-16
cost at iteration 195 :  7.00278327070477e-16
cost at iteration 196 :  6.83613029721906e-16
cost at iteration 197 :  6.679801795033252e-16
cost at it

cost at iteration 352 :  1.8314711799565176e-16
cost at iteration 353 :  1.8230875098764145e-16
cost at iteration 354 :  1.8216784146271312e-16
cost at iteration 355 :  1.81339020823104e-16
cost at iteration 356 :  1.8121296024402017e-16
cost at iteration 357 :  1.8039330452114586e-16
cost at iteration 358 :  1.8028165729302855e-16
cost at iteration 359 :  1.7947080185121214e-16
cost at iteration 360 :  1.7937314934829296e-16
cost at iteration 361 :  1.785707455413949e-16
cost at iteration 362 :  1.7848668525666622e-16
cost at iteration 363 :  1.776923996501491e-16
cost at iteration 364 :  1.7762154439381888e-16
cost at iteration 365 :  1.7683505803202424e-16
cost at iteration 366 :  1.7677703517575254e-16
cost at iteration 367 :  1.759980428912923e-16
cost at iteration 368 :  1.7595249365510846e-16
cost at iteration 369 :  1.751807034177286e-16
cost at iteration 370 :  1.7514728219703683e-16
cost at iteration 371 :  1.7438241449912228e-16
cost at iteration 372 :  1.7436078822912926e-1

cost at iteration 530 :  1.430961996574744e-16
cost at iteration 531 :  1.4252211984844159e-16
cost at iteration 532 :  1.4289919122142385e-16
cost at iteration 533 :  1.4232616482692161e-16
cost at iteration 534 :  1.4270461321865533e-16
cost at iteration 535 :  1.4213264186872628e-16
cost at iteration 536 :  1.4251241704678467e-16
cost at iteration 537 :  1.4194150281903701e-16
cost at iteration 538 :  1.4232255529253583e-16
cost at iteration 539 :  1.4175270069346309e-16
cost at iteration 540 :  1.4213498169797224e-16
cost at iteration 541 :  1.415661896449016e-16
cost at iteration 542 :  1.4194965112783957e-16
cost at iteration 543 :  1.4138192493145096e-16
cost at iteration 544 :  1.4176651953788478e-16
cost at iteration 545 :  1.4119986288540961e-16
cost at iteration 546 :  1.4158554394433304e-16
cost at iteration 547 :  1.4101996088324467e-16
cost at iteration 548 :  1.4140668239421488e-16
cost at iteration 549 :  1.4084217731654876e-16
cost at iteration 550 :  1.412298939367084